In [ ]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib 

IMPORT NEEDED LIBRARIES

In [1]:
import cv2  #openCV for images
import numpy as np   #numpy for arrayas
import os
from matplotlib import pyplot as plt  # matplotlib for viewing plots
import time
import mediapipe as mp    #mediapipe for pose detection and tracking

In [2]:
#mediapipe models

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #CHANGE IMAGE TO RGB
    image.flags.writeable = False
    results = model.process(image)  #process image using mediapipe model
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  #change image back to BGR
    return image, results  

In [4]:
"""
This function draw the pose landmarks on the images
"""
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    mp_drawing.DrawingSpec(color = (80, 110, 10), thickness=1, circle_radius=1),  #line color
    mp_drawing.DrawingSpec(color = (80, 110, 10), thickness=1, circle_radius=1)  #dot color
    )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
    mp_drawing.DrawingSpec(color = (80, 22, 10), thickness=2, circle_radius=4),  #line color
    mp_drawing.DrawingSpec(color = (80, 44, 10), thickness=2, circle_radius=2)  #dot color
    )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
    mp_drawing.DrawingSpec(color = (121, 22, 76), thickness=2, circle_radius=4),  #line color
    mp_drawing.DrawingSpec(color = (121, 44, 250), thickness=2, circle_radius=2)  #dot color
    )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
    mp_drawing.DrawingSpec(color = (245, 117, 66), thickness=2, circle_radius=4),  #line color
    mp_drawing.DrawingSpec(color = (245, 66, 230), thickness=2, circle_radius=2)  #dot color
    )

In [ ]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        rect, frame = cap.read()

        #make predictions
        image, results = mediapipe_detection(frame, holistic)
        print(results )

        #Draw Landmarks
        draw_styled_landmarks(image, results)

        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break 

cap.release()
cv2.destroyAllWindows()

Extract Keypoint Values

In [ ]:
pose = []

for point in results.pose_landmarks.landmark:
    test = np.array([point.x, point.y, point.z, point.visibility])
    pose.append(test)

In [ ]:
# refactored code

pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

face  = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404) 

lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3 ) 

In [6]:
def extract_keypoints(results):
    # refactored code

    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

    face  = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3) 

    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3 ) 

    return np.concatenate([pose, face, lh, rh])

In [ ]:
results.left_hand_landmarks.landmark 

SETUP FOLDERS

In [7]:
# path for exported data
DATA_PATH = os.path.join('./MP_Data')

# Action we try to detect
actions = np.array(['hello','thank you', 'iloveyou'])

# 30 videos worth of data
no_sequences = 30

# videos are going to be 30 frames in length
sequence_length = 30


In [9]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [12]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through Actions
    for action in actions:
        #loop through sequences
        for sequence in range(no_sequences):
            #loop through video length:
            for frame_num in range(sequence_length):


                rect, frame = cap.read()

                #make predictions
                image, results = mediapipe_detection(frame, holistic)
                #print(results )

                #Draw Landmarks
                draw_styled_landmarks(image, results)

                if frame_num == 0:
                    cv2.putText(image, "STARTING COLLECTION", (120, 200), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, "collecting frames for {} video Number {}".format(action, sequence), (15, 12), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, "collecting frames for {} video Number {}".format(action, sequence), (15, 12), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 0, 255), 1, cv2.LINE_AA)

                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                cv2.imshow('OpenCV Feed', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break 

    cap.release()
    cv2.destroyAllWindows()

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [15]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
X = np.array(sequences) 
y = to_categorical(labels).astype(int)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [18]:
#Train Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

In [19]:
#Create Log Directory for Tensorboard callbacks

log_dir = os.path.join('Logs')
tb_callbacks = TensorBoard(log_dir=log_dir)
early_stop = EarlyStopping(
    patience=10,
    min_delta=0.01,
    restore_best_weights = True
)

2021-11-03 19:54:22.516805: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-03 19:54:22.516830: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-03 19:54:22.518759: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


In [20]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

2021-11-03 19:55:10.622792: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
res = [.7, 0.2, 0.1]

In [22]:
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics = ['categorical_accuracy']
)

In [23]:
model.fit(
    X_train, y_train,
    epochs=2000,
    callbacks=[tb_callbacks, early_stop]
)

2021-11-03 19:55:26.125220: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/2000
2/3 [===================>..........] - ETA: 0s - loss: 1.1836 - categorical_accuracy: 0.3438

2021-11-03 19:55:29.581109: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-03 19:55:29.581129: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-03 19:55:29.661794: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-03 19:55:29.715625: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-03 19:55:29.768456: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: Logs/train/plugins/profile/2021_11_03_19_55_29



3/3 [==============================] - 4s 216ms/step - loss: 1.1717 - categorical_accuracy: 0.3529
Epoch 2/2000
1/3 [=========>....................] - ETA: 0s - loss: 1.2505 - categorical_accuracy: 0.1562

2021-11-03 19:55:29.798632: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to Logs/train/plugins/profile/2021_11_03_19_55_29/Eliass-MacBook-Pro.local.trace.json.gz
2021-11-03 19:55:29.842755: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: Logs/train/plugins/profile/2021_11_03_19_55_29

2021-11-03 19:55:29.843200: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to Logs/train/plugins/profile/2021_11_03_19_55_29/Eliass-MacBook-Pro.local.memory_profile.json.gz
2021-11-03 19:55:29.849482: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: Logs/train/plugins/profile/2021_11_03_19_55_29
Dumped tool data for xplane.pb to Logs/train/plugins/profile/2021_11_03_19_55_29/Eliass-MacBook-Pro.local.xplane.pb
Dumped tool data for overview_page.pb to Logs/train/plugins/profile/2021_11_03_19_55_29/Eliass-MacBook-Pro.local.ove

3/3 [==============================] - 0s 55ms/step - loss: 1.2048 - categorical_accuracy: 0.2235
Epoch 3/2000
3/3 [==============================] - 0s 54ms/step - loss: 1.5144 - categorical_accuracy: 0.3294
Epoch 4/2000
3/3 [==============================] - 0s 54ms/step - loss: 1.1480 - categorical_accuracy: 0.3294
Epoch 5/2000
3/3 [==============================] - 0s 51ms/step - loss: 1.0836 - categorical_accuracy: 0.3529
Epoch 6/2000
3/3 [==============================] - 0s 58ms/step - loss: 1.0933 - categorical_accuracy: 0.3529
Epoch 7/2000
3/3 [==============================] - 0s 54ms/step - loss: 1.0811 - categorical_accuracy: 0.3529
Epoch 8/2000
3/3 [==============================] - 0s 52ms/step - loss: 1.0656 - categorical_accuracy: 0.3529
Epoch 9/2000
3/3 [==============================] - 0s 55ms/step - loss: 1.0770 - categorical_accuracy: 0.4000
Epoch 10/2000
3/3 [==============================] - 0s 84ms/step - loss: 1.1070 - categorical_accuracy: 0.3176
Epoch 11/2000

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [25]:
res = model.predict(X_test)

In [32]:
actions[np.argmax(res[2])]

'thank you'

In [33]:
actions[np.argmax(y_test[2])]

'hello'

In [ ]:
#Save Weight

model.save('action_v1.h5')

In [34]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [40]:
yhat = model.predict(X_test)

In [41]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [37]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3., 0.],
        [2., 0.]],

       [[0., 5.],
        [0., 0.]],

       [[2., 0.],
        [3., 0.]]])

In [42]:
accuracy_score(ytrue, yhat)

0.8

In [10]:
from tensorflow.keras.models import load_model

model = load_model('action.h5')

model.summary()

2021-11-04 23:41:34.871208: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 64)            442112    
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
________________________________________________

In [11]:
from scipy import stats

In [12]:

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [13]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

2021-11-04 23:41:47.306426: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOu